In [2]:
import random

from dotenv import load_dotenv
load_dotenv('.env')

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType, initialize_agent
from langchain_core.tools import tool
from pydantic.v1 import BaseModel, Field

gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

/home/ubuntu/.cache/pypoetry/virtualenvs/langchain-demo-wAV8wxVh-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class UserInput(BaseModel):
    location: str = Field(..., title="Location", description="The location to fetch data.")

@tool("fetch_temperature", args_schema=UserInput)
def fetch_temperature(user_input: UserInput) -> int:
    """Fetches the temperature for a given location."""
    # return {'temperature': random.randint(15, 35), 'unit': 'celsius', 'location': location}
    return random.randint(15, 35)

@tool("fetch_weather", args_schema=UserInput)
def fetch_weather(user_input: UserInput) -> str:
    """Fetches the weather for a given location."""
    # return {'weather': random.choice(['sunny', 'cloudy', 'rainy']), 'location': location}
    return random.choice(['sunny', 'cloudy', 'rainy'])

custom_tools = [fetch_temperature, fetch_weather]

In [6]:
agent = initialize_agent(
    custom_tools, gemini_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)
agent.run("屏東的天氣跟溫度，給我穿著建議。")



> Entering new AgentExecutor chain...
我必須先取得屏東的天氣和溫度
Action: fetch_weather
Action Input: UserInput(location="屏東")
Observation: cloudy
Thought:我現在知道屏東的天氣是多雲
Action: fetch_temperature
Action Input: UserInput(location="屏東")
Observation: 22
Thought:我現在知道屏東的溫度是22度
Final Answer: 屏東的天氣多雲，溫度22度，穿著上建議洋裝或短袖襯衫，防曬用品別忘記。

> Finished chain.


'屏東的天氣多雲，溫度22度，穿著上建議洋裝或短袖襯衫，防曬用品別忘記。'